In [96]:
%matplotlib notebook
import cvxpy as cp
import dccp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.utils import shuffle
import matplotlib.patches as mpatches
import json
import random
import math
import os, psutil
from datetime import datetime

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
np.random.seed(0)

TRAIN_SLOPE = 2
EVAL_SLOPE = 5
X_LOWER_BOUND = -10
X_UPPER_BOUND = 10
SEED = 2

# Utils

In [97]:
def split_data(X, Y, percentage):
    num_val = int(len(X)*percentage)
    return X[num_val:], Y[num_val:], X[:num_val], Y[:num_val]

def shuffle(X, Y):
    torch.manual_seed(0)
    np.random.seed(0)
    data = torch.cat((Y, X), 1)
    data = data[torch.randperm(data.size()[0])]
    X = data[:, 1:]
    Y = data[:, 0]
    return X, Y

def conf_mat(Y1, Y2):
    num_of_samples = len(Y1)
    mat = confusion_matrix(Y1, Y2, labels=[-1, 1])*100/num_of_samples
    acc = np.trace(mat)
    return mat, acc

def calc_accuracy(Y, Ypred):
    num = len(Y)
    temp = Y - Ypred
    acc = len(temp[temp == 0])*1./num
    return acc

# CCP classes

In [98]:
class CCP:
    def __init__(self, x_dim, batch_size, funcs, scale):
        self.f_derivative = funcs["f_derivative"]
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable((batch_size, x_dim))
        self.xt = cp.Parameter((batch_size, x_dim))
        self.r = cp.Parameter((batch_size, x_dim))
        self.w = cp.Parameter(x_dim)
        self.b = cp.Parameter(1)
        self.slope = cp.Parameter(1)

        target = cp.diag(self.x@(self.f_derivative(self.xt, self.w, self.b, self.slope).T))-self.g(self.x, self.w, self.b, self.slope)-self.c(self.x, self.r, x_dim, scale)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        self.prob = cp.Problem(cp.Maximize(cp.sum(target)), constraints)
        
    def ccp(self, r):
        """
        numpy to numpy
        """
        self.xt.value = r
        self.r.value = r
        result = self.prob.solve()
        diff = np.linalg.norm(self.xt.value - self.x.value)
        cnt = 0
        while diff > 0.001 and cnt < 100:
            cnt += 1
            self.xt.value = self.x.value
            result = self.prob.solve()
            diff = np.linalg.norm(self.x.value - self.xt.value)
        return self.x.value
    
    def optimize_X(self, X, w, b, slope):
        """
        tensor to tensor
        """
        w = w.detach().numpy()
        b = b.detach().numpy()
        slope = np.full(1, slope)
        X = X.numpy()
        
        self.w.value = w
        self.b.value = b
        self.slope.value = slope
        return torch.from_numpy(self.ccp(X))
        # return torch.stack([torch.from_numpy(self.ccp(x)) for x in X])

In [99]:
class DELTA():
    
    def __init__(self, x_dim, funcs, scale):
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.b = cp.Parameter(1, value = np.random.randn(1))
        self.f_der = cp.Parameter(x_dim, value = np.random.randn(x_dim))

        target = self.x@self.f_der-self.g(self.x, self.w, self.b, TRAIN_SLOPE)-self.c(self.x, self.r, x_dim, scale)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        objective = cp.Maximize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.b, self.f_der],
                                variables=[self.x])
        
    def optimize_X(self, X, w, b, F_DER):
        return self.layer(X, w, b, F_DER)[0]

# Gain & Cost functions

In [100]:
def score(x, w, b):
    return x@w + b

def f(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) + 1)]), 2)

def g(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) - 1)]), 2)

def c(x, r, x_dim, scale):
    return (scale)*cp.sum_squares(x-r)

def f_derivative(x, w, b, slope):
    return 0.5*cp.multiply(slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1)), w)
    
def f_batch(x, w, b, slope):
    return 0.5*cp.norm(cp.vstack([np.ones(x.shape[0]), (slope*score(x, w, b) + 1)]), 2, axis=0)

def g_batch(x, w, b, slope):
    return 0.5*cp.norm(cp.vstack([np.ones((1, x.shape[0])), cp.reshape((slope*score(x, w, b) - 1), (1, x.shape[0]))]), 2, axis=0)

def c_batch(x, r, x_dim, scale):
    return (scale)*cp.square(cp.norm(x-r, 2, axis=1))

def f_derivative_batch(x, w, b, slope):
    nablas = 0.5*slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1))
    return cp.reshape(nablas, (nablas.shape[0], 1))@cp.reshape(w, (1, x.shape[1]))

# Model

In [101]:
class MyStrategicModel(torch.nn.Module):
    def __init__(self, x_dim, batch_size, funcs, funcs_batch, train_slope, eval_slope, scale, strategic=False):
        torch.manual_seed(0)
        np.random.seed(0)

        super(MyStrategicModel, self).__init__()
        self.x_dim = x_dim
        self.batch_size = batch_size
        self.train_slope, self.eval_slope = train_slope, eval_slope
        self.w = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(x_dim, dtype=torch.float64, requires_grad=True)))
        self.b = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(1, dtype=torch.float64, requires_grad=True)))
#         self.w = torch.nn.parameter.Parameter(torch.zeros(x_dim, dtype=torch.float64, requires_grad=True))
#         self.b = torch.nn.parameter.Parameter(torch.zeros(1, dtype=torch.float64, requires_grad=True))
        self.strategic = strategic
        self.ccp = CCP(x_dim, batch_size, funcs_batch, scale)
        self.delta = DELTA(x_dim, funcs, scale)
        self.ccp_time = 0
        self.total_time = 0

    def forward(self, X, evaluation=False):
        if self.strategic:
            if evaluation:
                t1 = time.time()
                XT = self.ccp.optimize_X(X, self.w, self.b, self.eval_slope)
                self.ccp_time += time.time()-t1
                X_opt = XT
            else:
                t1 = time.time()
                XT = self.ccp.optimize_X(X, self.w, self.b, self.train_slope)
                self.ccp_time += time.time()-t1
                F_DER = self.get_f_ders(XT, self.train_slope)
                X_opt = self.delta.optimize_X(X, self.w, self.b, F_DER) # Xopt should be equal to XT but we do it again for the gradients
            output = self.score(X_opt)
        else:
            output = self.score(X)        
        return output
    
    def optimize_X(self, X, evaluation=False):
        slope = self.eval_slope if evaluation else self.train_slope
        return self.ccp.optimize_X(X, self.w, self.b, slope)
    
    def normalize_weights(self):
        with torch.no_grad():
            norm = torch.sqrt(torch.sum(self.w**2) + self.b**2)
            self.w /= norm
            self.b /= norm

    def score(self, x):
        return x@self.w + self.b
    
    def get_f_ders(self, XT, slope):
        # return torch.stack([0.5*slope*((slope*self.score(xt) + 1)/torch.sqrt((slope*self.score(xt) + 1)**2 + 1))*self.w for xt in XT])
        nablas = 0.5*slope*((slope*self.score(XT) + 1)/torch.sqrt((slope*self.score(XT) + 1)**2 + 1))
        return torch.reshape(nablas, (len(nablas), 1))@torch.reshape(self.w, (1, len(self.w)))

    def calc_accuracy(self, Y, Y_pred):
        Y_pred = torch.sign(Y_pred)
        num = len(Y)
        temp = Y - Y_pred
        acc = len(temp[temp == 0])*1./num        
        return acc
    
    def evaluate(self, X, Y):      
        return self.calc_accuracy(Y, self.forward(X, evaluation=True))
    
    def loss(self, Y, Y_pred):
        return torch.mean(torch.clamp(1 - Y_pred * Y, min=0))
    
    def save_model(self, train_errors, val_errors, train_losses, val_losses, info, path, comment=None):
        if comment is not None:
            path += "/" + comment
            
        filename = path + "/model.pt"
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        torch.save(self.state_dict(), filename)
                
        pd.DataFrame(np.array(train_errors)).to_csv(path + '/train_errors.csv')
        pd.DataFrame(np.array(val_errors)).to_csv(path + '/val_errors.csv')
        pd.DataFrame(np.array(train_losses)).to_csv(path + '/train_losses.csv')
        pd.DataFrame(np.array(val_losses)).to_csv(path + '/val_losses.csv')
        
        with open(path + "/info.txt", "w") as f:
            f.write(info)
    
    def load_model(self, filename):
        self.load_state_dict(torch.load(filename))
        self.eval()
    
    def fit(self, path, X, Y, Xval, Yval, opt, opt_kwargs={"lr":1e-3}, batch_size=128, epochs=100, verbose=False, callback=None, comment=None):
        train_dset = TensorDataset(X, Y)
        train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        test_dset = TensorDataset(Xval, Yval)
        test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=True)
        
        opt = opt(self.parameters(), **opt_kwargs)

        train_losses = []
        val_losses = []
        train_errors = []
        val_errors = []
        
        best_val_error = 1
        consecutive_no_improvement = 0

        total_time = time.time()
        for epoch in range(epochs):
            t1 = time.time()
            batch = 1
            train_losses.append([])
            train_errors.append([])
            for Xbatch, Ybatch in train_loader:
#                 try:
                opt.zero_grad()
                Ybatch_pred = self.forward(Xbatch)
                l = self.loss(Ybatch, Ybatch_pred)
                l.backward()
                opt.step()
                train_losses[-1].append(l.item())
                with torch.no_grad():
                    e = self.calc_accuracy(Ybatch, Ybatch_pred)
                    train_errors[-1].append(1-e)
                if verbose:
                    print("batch %03d / %03d | loss: %3.5f | err: %3.5f" %
                          (batch, len(train_loader), np.mean(train_losses[-1]), np.mean(train_errors[-1])))
                batch += 1
                if callback is not None:
                    callback()
#                 except:
#                     print("failed")
                
            with torch.no_grad():
                total_loss = 0
                total_error = 0
                batch = 0
                for Xbatch, Ybatch in test_loader:
#                     try:
                    Yval_pred = self.forward(Xbatch, evaluation=True)
                    val_loss = self.loss(Ybatch, Yval_pred).item()
                    total_loss += val_loss
                    val_error = 1-self.calc_accuracy(Ybatch, Yval_pred)
                    total_error += val_error
                    batch += 1
#                     except:
#                         print("failed")
                        
                avg_loss = total_loss/batch
                avg_error = total_error/batch
                val_losses.append(avg_loss)
                val_errors.append(avg_error)
                if avg_error < best_val_error:
                        consecutive_no_improvement = 0
                        best_val_error = avg_error
                        info = "training time in seconds: {}\nepoch: {}\nbatch size: {}\ntrain slope: {}\neval slope: {}\nlearning rate: {}\nvalidation loss: {}\nvalidation error: {}\n".format(
                        time.time()-total_time, epoch, batch_size, self.train_slope, self.eval_slope, opt_kwargs["lr"], avg_loss, avg_error)
                        self.save_model(train_errors, val_errors, train_losses, val_losses, info, path, comment)
                        print("model saved!")

                else:
                    consecutive_no_improvement += 1
                    if consecutive_no_improvement >= 4:
                        break
                    
            t2 = time.time()
            if verbose:
                print("------------- epoch %03d / %03d | time: %03d sec | loss: %3.5f | err: %3.5f" % (epoch + 1, epochs, t2-t1, val_losses[-1], val_errors[-1]))
        
        self.total_time = time.time()-total_time
        print("training time: {} seconds".format(self.total_time)) 
        return train_errors, val_errors, train_losses, val_losses

In [102]:
def gen_sklearn_data(x_dim, N, informative_frac=1, shift_range=1, scale_range=1, noise_frac=0.01):
    torch.manual_seed(0)
    np.random.seed(0)
    n_informative = int(informative_frac*x_dim)
    n_redundant = x_dim - n_informative
    shift_arr = shift_range*np.random.randn(x_dim)
    scale_arr = scale_range*np.random.randn(x_dim)
    X, Y = make_classification(n_samples=N, n_features=x_dim, n_informative=n_informative, n_redundant=n_redundant,
                               flip_y=noise_frac, shift=shift_arr, scale=scale_arr, random_state=0)
    Y[Y == 0] = -1
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_spam_data():
    torch.manual_seed(0)
    np.random.seed(0)
    path = r"C:\Users\sagil\Desktop\nir_project\tip_spam_data\IS_journal_tip_spam.arff"
    data, meta = arff.loadarff(path)
    df = pd.DataFrame(data)
    most_disc = ['qTips_plc', 'rating_plc', 'qEmail_tip', 'qContacts_tip', 'qURL_tip', 'qPhone_tip', 'qNumeriChar_tip', 'sentistrength_tip', 'combined_tip', 'qWords_tip', 'followers_followees_gph', 'qunigram_avg_tip', 'qTips_usr', 'indeg_gph', 'qCapitalChar_tip', 'class1']
    df = df[most_disc]
    df["class1"].replace({b'spam': -1, b'notspam': 1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y = df['class1'].values
    X = df.drop('class1', axis = 1).values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_card_fraud_data():
    torch.manual_seed(0)
    np.random.seed(0)
    df = pd.read_csv('C:/Users/sagil/Desktop/nir_project/card_fraud_dataset/creditcard.csv')

    rob_scaler = RobustScaler()

    df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
    df.drop(['Time','Amount'], axis=1, inplace=True)
    scaled_amount = df['scaled_amount']
    df.drop(['scaled_amount'], axis=1, inplace=True)
    df.insert(0, 'scaled_amount', scaled_amount)

    df["Class"].replace({1: -1, 0: 1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    # amount of fraud classes 492 rows.
    fraud_df = df.loc[df['Class'] == -1]
    non_fraud_df = df.loc[df['Class'] == 1][:492]

    normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

    # Shuffle dataframe rows
    df = normal_distributed_df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y = df['Class'].values
    X = df.drop('Class', axis = 1).values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_credit_default_data():
    torch.manual_seed(0)
    np.random.seed(0)
    url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
    df = pd.read_csv(url)
    df["NoDefaultNextMonth"].replace({0: -1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    df = df.drop(['Married', 'Single', 'Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60'], axis = 1)

    fraud_df = df.loc[df["NoDefaultNextMonth"] == -1]
    non_fraud_df = df.loc[df["NoDefaultNextMonth"] == 1][:6636]

    normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

    # Shuffle dataframe rows
    df = normal_distributed_df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    
    scaler = StandardScaler()
    df.loc[:, df.columns != "NoDefaultNextMonth"] = scaler.fit_transform(df.drop("NoDefaultNextMonth", axis=1)) 
    Y, X = df.iloc[:, 0].values, df.iloc[:, 1:].values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_financial_distress_data():
    torch.manual_seed(0)
    np.random.seed(0)
    data = pd.read_csv("C:/Users/sagil/Desktop/nir_project/financial_distress_data/Financial Distress.csv")

    data = data[data.columns.drop(list(data.filter(regex='x80')))] # Since it is a categorical feature with 37 features.
    x_dim = len(data.columns) - 3
    data.drop(['Time'], axis=1, inplace=True)

    data_grouped = data.groupby(['Company']).last()

    scaler = StandardScaler()
    data_grouped.loc[:, data_grouped.columns != "Financial Distress"] = scaler.fit_transform(data_grouped.drop("Financial Distress", axis=1))

    # Shuffle dataframe rows
    data_grouped = data_grouped.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y, X = data_grouped.iloc[:, 0].values, data_grouped.iloc[:, 1:].values
    for y in range(0,len(Y)): # Coverting target variable from continuous to binary form
        if Y[y] < -0.5:
              Y[y] = -1
        else:
              Y[y] = 1
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

In [ ]:
path = "C:/Users/sagil/Desktop/nir_project/models/runtime_varying_batch_size"
epochs = 5
x_dim = 10
scale = 1
X, Y = gen_sklearn_data(x_dim, 1024)
X, Y, Xval, Yval = split_data(X, Y, 0.25)
print(Xval.size())
print("percent of positive samples: {}%".format(100 * len(Y[Y == 1]) / len(Y)))

funcs = {"f": f, "g": g, "f_derivative": f_derivative, "c": c, "score": score}
funcs_batch = {"f": f_batch, "g": g_batch, "f_derivative": f_derivative_batch, "c": c_batch, "score": score}

total = []
ccp = []
for batch_size in (2**np.arange(9)).tolist():
    strategic_model = MyStrategicModel(x_dim, batch_size, funcs, funcs_batch, TRAIN_SLOPE, EVAL_SLOPE, scale=scale, strategic=True)
    strategic_model.fit(path, X, Y, Xval, Yval,
                        opt=torch.optim.Adam, opt_kwargs={"lr": (1e-1)},
                        batch_size=batch_size, epochs=epochs, verbose=True,
                       comment="batched")
    
    total_time = strategic_model.total_time
    ccp_time = strategic_model.ccp_time
    total.append(total_time)
    ccp.append(ccp_time)
    pd.DataFrame(np.array(total)).to_csv(path + '/total_timing_results.csv')
    pd.DataFrame(np.array(ccp)).to_csv(path + '/ccp_timing_results.csv')

torch.Size([256, 10])
percent of positive samples: 52.473958333333336%
batch 001 / 768 | loss: 1.60883 | err: 1.00000
batch 002 / 768 | loss: 1.42408 | err: 1.00000


C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\l

batch 003 / 768 | loss: 1.43130 | err: 1.00000
batch 004 / 768 | loss: 1.46224 | err: 1.00000
batch 005 / 768 | loss: 1.31990 | err: 0.80000
batch 006 / 768 | loss: 1.22072 | err: 0.66667
batch 007 / 768 | loss: 1.19027 | err: 0.71429
batch 008 / 768 | loss: 1.20005 | err: 0.75000
batch 009 / 768 | loss: 1.10415 | err: 0.66667
batch 010 / 768 | loss: 1.11612 | err: 0.70000
batch 011 / 768 | loss: 1.06212 | err: 0.63636
batch 012 / 768 | loss: 0.97361 | err: 0.58333
batch 013 / 768 | loss: 0.90332 | err: 0.53846
batch 014 / 768 | loss: 0.89250 | err: 0.50000
batch 015 / 768 | loss: 0.83830 | err: 0.46667
batch 016 / 768 | loss: 0.78591 | err: 0.43750
batch 017 / 768 | loss: 0.73968 | err: 0.41176
batch 018 / 768 | loss: 0.86659 | err: 0.44444
batch 019 / 768 | loss: 0.91253 | err: 0.47368
batch 020 / 768 | loss: 0.87997 | err: 0.45000
batch 021 / 768 | loss: 0.83807 | err: 0.42857
batch 022 / 768 | loss: 0.80810 | err: 0.40909
batch 023 / 768 | loss: 0.77296 | err: 0.39130
batch 024 / 7

batch 180 / 768 | loss: 0.75600 | err: 0.34444
batch 181 / 768 | loss: 0.75182 | err: 0.34254
batch 182 / 768 | loss: 0.74769 | err: 0.34066
batch 183 / 768 | loss: 0.75875 | err: 0.34426
batch 184 / 768 | loss: 0.76934 | err: 0.34783
batch 185 / 768 | loss: 0.76518 | err: 0.34595
batch 186 / 768 | loss: 0.76107 | err: 0.34409
batch 187 / 768 | loss: 0.76735 | err: 0.34759
batch 188 / 768 | loss: 0.77262 | err: 0.35106
batch 189 / 768 | loss: 0.78713 | err: 0.35450
batch 190 / 768 | loss: 0.78299 | err: 0.35263
batch 191 / 768 | loss: 0.78634 | err: 0.35602
batch 192 / 768 | loss: 0.78987 | err: 0.35938
batch 193 / 768 | loss: 0.79312 | err: 0.36269
batch 194 / 768 | loss: 0.79792 | err: 0.36598
batch 195 / 768 | loss: 0.80408 | err: 0.36923
batch 196 / 768 | loss: 0.80297 | err: 0.36735
batch 197 / 768 | loss: 0.80891 | err: 0.37056
batch 198 / 768 | loss: 0.80874 | err: 0.36869
batch 199 / 768 | loss: 0.80468 | err: 0.36683
batch 200 / 768 | loss: 0.80066 | err: 0.36500
batch 201 / 7

batch 355 / 768 | loss: 0.75240 | err: 0.34085
batch 356 / 768 | loss: 0.75029 | err: 0.33989
batch 357 / 768 | loss: 0.74819 | err: 0.33894
batch 358 / 768 | loss: 0.74610 | err: 0.33799
batch 359 / 768 | loss: 0.75219 | err: 0.33983
batch 360 / 768 | loss: 0.75916 | err: 0.34167
batch 361 / 768 | loss: 0.76122 | err: 0.34349
batch 362 / 768 | loss: 0.76400 | err: 0.34530
batch 363 / 768 | loss: 0.76309 | err: 0.34435
batch 364 / 768 | loss: 0.76099 | err: 0.34341
batch 365 / 768 | loss: 0.76471 | err: 0.34521
batch 366 / 768 | loss: 0.76739 | err: 0.34699
batch 367 / 768 | loss: 0.76530 | err: 0.34605
batch 368 / 768 | loss: 0.76322 | err: 0.34511
batch 369 / 768 | loss: 0.76605 | err: 0.34688
batch 370 / 768 | loss: 0.76531 | err: 0.34595
batch 371 / 768 | loss: 0.76408 | err: 0.34501
batch 372 / 768 | loss: 0.76377 | err: 0.34409
batch 373 / 768 | loss: 0.76172 | err: 0.34316
batch 374 / 768 | loss: 0.76346 | err: 0.34492
batch 375 / 768 | loss: 0.76272 | err: 0.34400
batch 376 / 7

...

batch 054 / 096 | loss: 0.71072 | err: 0.36806
batch 055 / 096 | loss: 0.71043 | err: 0.36818
batch 056 / 096 | loss: 0.70736 | err: 0.36607
batch 057 / 096 | loss: 0.71025 | err: 0.36623
batch 058 / 096 | loss: 0.70566 | err: 0.36638
batch 059 / 096 | loss: 0.69737 | err: 0.36017
batch 060 / 096 | loss: 0.69166 | err: 0.35625
batch 061 / 096 | loss: 0.68570 | err: 0.35246
batch 062 / 096 | loss: 0.68236 | err: 0.35081
batch 063 / 096 | loss: 0.69276 | err: 0.35714
batch 064 / 096 | loss: 0.69003 | err: 0.35742
batch 065 / 096 | loss: 0.68457 | err: 0.35385
batch 066 / 096 | loss: 0.68466 | err: 0.35417
batch 067 / 096 | loss: 0.67912 | err: 0.35261
batch 068 / 096 | loss: 0.67932 | err: 0.35294
batch 069 / 096 | loss: 0.67579 | err: 0.35145
batch 070 / 096 | loss: 0.68553 | err: 0.35357
batch 071 / 096 | loss: 0.69155 | err: 0.35563
batch 072 / 096 | loss: 0.68481 | err: 0.35069
batch 073 / 096 | loss: 0.68019 | err: 0.34932
batch 074 / 096 | loss: 0.68395 | err: 0.35135
batch 075 / 0

batch 033 / 096 | loss: 0.75300 | err: 0.38258
batch 034 / 096 | loss: 0.74366 | err: 0.37868
batch 035 / 096 | loss: 0.74586 | err: 0.37857
batch 036 / 096 | loss: 0.74583 | err: 0.37153
batch 037 / 096 | loss: 0.75431 | err: 0.37838
batch 038 / 096 | loss: 0.75601 | err: 0.37829
batch 039 / 096 | loss: 0.75569 | err: 0.38141
batch 040 / 096 | loss: 0.75023 | err: 0.37812
batch 041 / 096 | loss: 0.74393 | err: 0.37500
batch 042 / 096 | loss: 0.74216 | err: 0.37798
batch 043 / 096 | loss: 0.73476 | err: 0.37500
batch 044 / 096 | loss: 0.73674 | err: 0.37784
batch 045 / 096 | loss: 0.72496 | err: 0.36944
batch 046 / 096 | loss: 0.72680 | err: 0.36957
batch 047 / 096 | loss: 0.72597 | err: 0.36702
batch 048 / 096 | loss: 0.72978 | err: 0.36979
batch 049 / 096 | loss: 0.72515 | err: 0.36735
batch 050 / 096 | loss: 0.72052 | err: 0.36250
batch 051 / 096 | loss: 0.73554 | err: 0.36765
batch 052 / 096 | loss: 0.73314 | err: 0.36538
batch 053 / 096 | loss: 0.72789 | err: 0.36085
batch 054 / 0

batch 012 / 048 | loss: 0.83261 | err: 0.40104
batch 013 / 048 | loss: 0.86307 | err: 0.41346
batch 014 / 048 | loss: 0.85845 | err: 0.42411
batch 015 / 048 | loss: 0.85537 | err: 0.42083
batch 016 / 048 | loss: 0.83367 | err: 0.41016
batch 017 / 048 | loss: 0.83172 | err: 0.41544
batch 018 / 048 | loss: 0.82203 | err: 0.40972
batch 019 / 048 | loss: 0.81618 | err: 0.41118
batch 020 / 048 | loss: 0.79036 | err: 0.40000
batch 021 / 048 | loss: 0.77189 | err: 0.38988
batch 022 / 048 | loss: 0.75964 | err: 0.38068
batch 023 / 048 | loss: 0.76059 | err: 0.38315
batch 024 / 048 | loss: 0.76092 | err: 0.38021
batch 025 / 048 | loss: 0.75803 | err: 0.38000
batch 026 / 048 | loss: 0.75035 | err: 0.37500
batch 027 / 048 | loss: 0.75444 | err: 0.37500
batch 028 / 048 | loss: 0.75346 | err: 0.37723
batch 029 / 048 | loss: 0.75503 | err: 0.37500
batch 030 / 048 | loss: 0.75744 | err: 0.37708
batch 031 / 048 | loss: 0.75711 | err: 0.37903
batch 032 / 048 | loss: 0.74410 | err: 0.37109
batch 033 / 0

batch 037 / 048 | loss: 0.68212 | err: 0.33953
batch 038 / 048 | loss: 0.68088 | err: 0.34046
batch 039 / 048 | loss: 0.67869 | err: 0.33814
batch 040 / 048 | loss: 0.67500 | err: 0.33437
batch 041 / 048 | loss: 0.67628 | err: 0.33384
batch 042 / 048 | loss: 0.67513 | err: 0.33333
batch 043 / 048 | loss: 0.67264 | err: 0.32994
batch 044 / 048 | loss: 0.66642 | err: 0.32670
batch 045 / 048 | loss: 0.66726 | err: 0.32778
batch 046 / 048 | loss: 0.66872 | err: 0.32609
batch 047 / 048 | loss: 0.66793 | err: 0.32580
batch 048 / 048 | loss: 0.66331 | err: 0.32292
------------- epoch 004 / 005 | time: 029 sec | loss: 0.74580 | err: 0.37500
batch 001 / 048 | loss: 0.75050 | err: 0.37500
batch 002 / 048 | loss: 0.88819 | err: 0.50000
batch 003 / 048 | loss: 0.81576 | err: 0.41667
batch 004 / 048 | loss: 0.76209 | err: 0.39062
batch 005 / 048 | loss: 0.66555 | err: 0.33750
batch 006 / 048 | loss: 0.61986 | err: 0.30208
batch 007 / 048 | loss: 0.60673 | err: 0.29464
batch 008 / 048 | loss: 0.6727

batch 009 / 024 | loss: 0.72545 | err: 0.40278
batch 010 / 024 | loss: 0.72191 | err: 0.40000
batch 011 / 024 | loss: 0.71662 | err: 0.39489
batch 012 / 024 | loss: 0.70891 | err: 0.39062
batch 013 / 024 | loss: 0.69743 | err: 0.37740
batch 014 / 024 | loss: 0.68656 | err: 0.36384
batch 015 / 024 | loss: 0.67271 | err: 0.35417
batch 016 / 024 | loss: 0.65893 | err: 0.34570
batch 017 / 024 | loss: 0.67484 | err: 0.35478
batch 018 / 024 | loss: 0.69362 | err: 0.36632
batch 019 / 024 | loss: 0.68996 | err: 0.36513
batch 020 / 024 | loss: 0.68067 | err: 0.35938
batch 021 / 024 | loss: 0.68283 | err: 0.36012
batch 022 / 024 | loss: 0.67829 | err: 0.35653
batch 023 / 024 | loss: 0.67137 | err: 0.35326
batch 024 / 024 | loss: 0.67153 | err: 0.35286
------------- epoch 005 / 005 | time: 025 sec | loss: 0.71739 | err: 0.35547
training time: 140.9406497478485 seconds
batch 001 / 012 | loss: 0.97793 | err: 0.54688
batch 002 / 012 | loss: 0.89142 | err: 0.43750
batch 003 / 012 | loss: 0.85175 | er

In [48]:
training_datas = []

# distress 
X, Y = load_financial_distress_data()
X, Y, Xval, Yval = split_data(X, Y, 0.25)
n = len(X)
effective_n = n - n%24
X, Y = X[:effective_n], Y[:effective_n]
n = len(Xval)
effective_n = n - n%24
Xval, Yval = Xval[:effective_n], Yval[:effective_n]
training_datas.append({"X": X,
                        "Y": Y,
                        "Xval": Xval,
                        "Yval": Yval,
                        "epochs": 7,
                        "batch_size": 24,
                        "name": "distress"})

# fraud dataset
X, Y = load_card_fraud_data()
X, Y, Xval, Yval = split_data(X, Y, 0.25)
n = len(X)
effective_n = n - n%24
X, Y = X[:effective_n], Y[:effective_n]
n = len(Xval)
effective_n = n - n%24
Xval, Yval = Xval[:effective_n], Yval[:effective_n]
training_datas.append({"X": X,
                        "Y": Y,
                        "Xval": Xval,
                        "Yval": Yval,
                        "epochs": 7,
                        "batch_size": 24, 
                        "name": "fraud"})


# credit data
X, Y = load_credit_default_data()
X, Y = X[:3000], Y[:3000]
X, Y, Xval, Yval = split_data(X, Y, 0.25)
n = len(X)
effective_n = n - n%24
X, Y = X[:effective_n], Y[:effective_n]
n = len(Xval)
effective_n = n - n%24
Xval, Yval = Xval[:effective_n], Yval[:effective_n]
training_datas.append({"X": X,
                        "Y": Y,
                        "Xval": Xval,
                        "Yval": Yval,
                        "epochs": 7,
                        "batch_size": 64, 
                        "name": "credit"})

# spam dataset
X, Y = load_spam_data()
X, Y, Xval, Yval = split_data(X, Y, 0.25)
n = len(X)
effective_n = n - n%24
X, Y = X[:effective_n], Y[:effective_n]
n = len(Xval)
effective_n = n - n%24
Xval, Yval = Xval[:effective_n], Yval[:effective_n]
training_datas.append({"X": X,
                        "Y": Y,
                        "Xval": Xval,
                        "Yval": Yval,
                        "epochs": 7,
                        "batch_size": 128, 
                        "name": "spam"})

In [ ]:
PATH = "C:/Users/sagil/Desktop/nir_project/models/real_dataset_runtimes"

for training_data in training_datas:
    path = PATH + "/" + training_data["name"]
    
    # load dataset
    X = training_data["X"]
    Y = training_data["Y"]
    Xval = training_data["Xval"]
    Yval = training_data["Yval"]
    
    # save dataset splits
    if not os.path.exists(path):
        os.makedirs(path)
    pd.DataFrame(X.numpy()).to_csv(path + '/X.csv')
    pd.DataFrame(Y.numpy()).to_csv(path + '/Y.csv')
    pd.DataFrame(Xval.numpy()).to_csv(path + '/Xval.csv')
    pd.DataFrame(Yval.numpy()).to_csv(path + '/Yval.csv')
    
    # training parameters
    x_dim = len(X[0])
    epochs = training_data["epochs"]
    batch_size = training_data["batch_size"]

    funcs = {"f": f, "g": g, "f_derivative": f_derivative, "c": c, "score": score}
    funcs_batch = {"f": f_batch, "g": g_batch, "f_derivative": f_derivative_batch, "c": c_batch, "score": score}


    strategic_model = MyStrategicModel(x_dim, batch_size, funcs, funcs_batch, TRAIN_SLOPE, EVAL_SLOPE, scale=scale, strategic=True)
    strategic_model.fit(path, X, Y, Xval, Yval,
                opt=torch.optim.Adam, opt_kwargs={"lr": 5*(1e-1)},
                batch_size=batch_size, epochs=epochs, verbose=True,
               comment="batched")
    
    runtimes = [strategic_model.total_time, strategic_model.ccp_time]     
    pd.DataFrame(np.array(runtimes)).to_csv(path + '/results.csv')